### Computational Tests

In [1]:
from cobra.io import load_json_model
import pandas as pd

# model = read_sbml_model('iCHOv3_CHO_05072023.xml')
model = load_json_model('iCHOv3_CHO_05072023.json')

#### Blocked reactions and Dead-End metabolites

#### Auxotrophies

#### Matrix Sparsity

#### Gene Deletion test

##### Essential for growth genes and comparison to (An optimised genome-wide, virus free CRISPR screen for mammalian cells)

##### Commparsion to other models

#### Biomass prediction using exp. data (TODO)